# Lab 01 · FastAPI, Vite, and Git

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- A FastAPI backend is scaffolded with health and echo routes.
- A Vite React frontend is outlined with modern tooling defaults.
- A Git repository is initialized with environment templates.

## What will be learned
- The structure of a basic FastAPI service is reviewed.
- Development-time CORS settings are configured.
- Vite-powered React scaffolding steps are rehearsed.
- Git initialization practices are reinforced.

### FastAPI framework primer

[FastAPI](https://fastapi.tiangolo.com/) is an asynchronous Python web framework that automatically binds request paths to function-based *path operations*. Each route can validate and serialize data with [Pydantic](https://docs.pydantic.dev/) models, so AI inference inputs and outputs are typed and self-documenting. Dependency injection simplifies connecting shared resources—like model clients, vector databases, or background task queues—while native async support keeps model calls non-blocking. FastAPI also ships with autogenerated Swagger and ReDoc interfaces, making it easy to explore and test AI endpoints as they evolve.

When building AI services, these features let you:
- Define explicit request/response schemas for prompts, embeddings, or chat histories.
- Stream long-running inference responses via async generators or `StreamingResponse`.
- Inject authenticated model clients or caching layers per request without manual wiring.
- Expose health checks and observability routes that operations teams can monitor.

### Vite + React framework primer

The frontend labs rely on [Vite](https://vitejs.dev/guide/) to scaffold a React UI with an ultra-fast dev server and on-demand ES module bundling. Vite provides hot module replacement (HMR), TypeScript-ready build tooling, and `.env` handling so secrets such as API base URLs or model keys stay out of source control. React components (see the [React documentation](https://react.dev/)) manage AI UI state—prompt inputs, streaming outputs, retry buttons—while hooks encapsulate fetch logic and error handling.

Within this stack you can:
- Organize feature folders (`src/features/chat`, `src/components/shared`) that mirror backend routes.
- Centralize API utilities that call FastAPI endpoints with `fetch` or libraries like Axios.
- Use React Query or custom hooks for polling, retries, and optimistic updates around AI inference.
- Leverage Vite plugins to proxy requests to the FastAPI server during local development, reducing CORS friction.



## Prerequisites & install
The container workflow depends on these locally installed tools:

```bash
docker --version
docker compose version
git --version
```

After cloning the repository, build and start the services with Docker Compose:

```bash
cd ai-web
docker compose build
docker compose up
# Backend → http://localhost:8000
# Frontend → http://localhost:5173

# Shut the stack down when finished exploring:
docker compose down
```

## Step-by-step tasks
Each step configures local source files that the Docker stack will mount so secrets remain outside the built images.

### Step 1: Backend folder layout and Dockerfile
A backend folder is created with starter FastAPI files and a Dockerfile that installs dependencies inside the image.

In [ ]:
from pathlib import Path
base = Path("ai-web/backend")
base.mkdir(parents=True, exist_ok=True)

# Ensure the application package exists so FastAPI can locate modules.
(base / "app").mkdir(parents=True, exist_ok=True)
(base / "app" / "__init__.py").write_text("")

# Provide a sample environment file that documents required secrets.
(base / ".env.example").write_text('# Environment variables (never commit real keys)GEMINI_API_KEY=')

# List Python dependencies that will be installed within the backend container.
(base / "requirements.txt").write_text('''fastapi
uvicorn[standard]
pydantic
python-dotenv
google-genai
faiss-cpu
numpy
''')

# Create the FastAPI entrypoint with health and echo routes.
(base / "app" / "main.py").write_text('''from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:5173"],
    allow_methods=["*"],
    allow_headers=["*"],
)


class EchoIn(BaseModel):
    msg: str


@app.get("/health")
def health():
    return {"status": "ok"}


@app.post("/echo")
def echo(payload: EchoIn):
    return {"msg": payload.msg}
''')

# Define a Dockerfile that installs dependencies and starts uvicorn.
(base / "Dockerfile").write_text('''FROM python:3.11-slim

WORKDIR /app

ENV PYTHONDONTWRITEBYTECODE=1 \n    PYTHONUNBUFFERED=1

COPY requirements.txt ./
RUN pip install --no-cache-dir -r requirements.txt

COPY app ./app

CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000"]
''')

print("Backend scaffold and Dockerfile were written under ai-web/backend.")

### Step 2: Frontend placeholders and Dockerfile
Frontend placeholders are positioned so Vite React components (`src/App.jsx`, `src/main.jsx`) can be customized while the generated `vite.config.js` continues to manage dev server defaults. A lightweight Dockerfile installs Node.js dependencies for the Vite dev server.

In [ ]:
from pathlib import Path
frontend = Path("ai-web/frontend")
src = frontend / "src"
frontend.mkdir(parents=True, exist_ok=True)
src.mkdir(parents=True, exist_ok=True)

# Create a lib directory for shared utilities between components.
(src / "lib").mkdir(parents=True, exist_ok=True)

# Bootstrap a simple API helper with descriptive error handling.
(src / "lib" / "api.js").write_text('''const BASE = import.meta.env.VITE_API_BASE || "http://localhost:8000";

export async function post(path, body) {
  const res = await fetch(`${BASE}${path}`, {
    method: 'POST',
    headers: { 'Content-Type': 'application/json' },
    body: JSON.stringify(body)
  });
  if (!res.ok) {
    throw new Error(`HTTP ${res.status}`);
  }
  return res.json();
}
''')

# Author a basic demo interface that interacts with the echo endpoint.
(src / "App.jsx").write_text('''import { useState } from 'react';
import { post } from './lib/api';

function App() {
  const [msg, setMsg] = useState('hello');
  const [response, setResponse] = useState('');
  const [loading, setLoading] = useState(false);
  const [error, setError] = useState('');

  async function handleSend() {
    setLoading(true);
    setError('');
    try {
      const json = await post('/echo', { msg });
      setResponse(json.msg);
    } catch (err) {
      setError(String(err));
    } finally {
      setLoading(false);
    }
  }

  return (
    <main style={{ padding: 24 }}>
      <h1>Lab 1 — Echo demo</h1>
      <input value={msg} onChange={(event) => setMsg(event.target.value)} />
      <button onClick={handleSend} disabled={loading}>Send</button>
      {loading && <p>Loading…</p>}
      {error && <p style={{ color: 'red' }}>{error}</p>}
      <pre>{response}</pre>
    </main>
  );
}

export default App;
''')

# Mount the App component using Vite's modern entry point.
(src / "main.jsx").write_text('''import React from 'react';
import ReactDOM from 'react-dom/client';
import App from './App.jsx';

ReactDOM.createRoot(document.getElementById('root')).render(
  <React.StrictMode>
    <App />
  </React.StrictMode>,
);
''')

# Ensure Vite configuration and HTML entrypoint exist for the dev server.
(frontend / "vite.config.js").write_text('''import { defineConfig } from "vite";
import react from "@vitejs/plugin-react";

export default defineConfig({
  plugins: [react()],
  server: {
    host: true,
    port: 5173,
  },
});
''')

(frontend / "index.html").write_text('''<!doctype html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <link rel="icon" type="image/svg+xml" href="/vite.svg" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Lab 1 Echo Demo</title>
  </head>
  <body>
    <div id="root"></div>
    <script type="module" src="/src/main.jsx"></script>
  </body>
</html>
''')

# Provide a package.json that mirrors the Vite React starter dependencies.
(frontend / "package.json").write_text('''{
  "name": "frontend",
  "version": "0.0.0",
  "private": true,
  "type": "module",
  "scripts": {
    "dev": "vite",
    "build": "vite build",
    "preview": "vite preview --host"
  },
  "dependencies": {
    "react": "^18.3.1",
    "react-dom": "^18.3.1"
  },
  "devDependencies": {
    "@vitejs/plugin-react": "^4.3.1",
    "vite": "^5.2.0"
  }
}
''')

# Provide a Dockerfile that installs dependencies and runs the Vite dev server.
(frontend / "Dockerfile").write_text('''FROM node:20-alpine

WORKDIR /app

COPY package*.json ./
RUN npm install

COPY . .

CMD ["npm", "run", "dev", "--", "--host"]
''')

print("Frontend placeholders and Dockerfile were written under ai-web/frontend.")

### Step 3: Docker Compose orchestration
A top-level `docker-compose.yml` is created so the backend and frontend can be started together with a single command.

In [ ]:
from pathlib import Path
compose = Path("ai-web/docker-compose.yml")
compose.parent.mkdir(parents=True, exist_ok=True)

# Define services that mount local source for rapid iteration while keeping dependencies inside the containers.
compose.write_text('''version: "3.9"

services:
  backend:
    build: ./backend
    ports:
      - "8000:8000"
    environment:
      - GEMINI_API_KEY=${GEMINI_API_KEY:-}
    volumes:
      - ./backend/app:/app/app
      - ./backend/.env.example:/app/.env:ro

  frontend:
    build: ./frontend
    ports:
      - "5173:5173"
    environment:
      - VITE_API_BASE=http://localhost:8000
    volumes:
      - ./frontend/src:/app/src
      - ./frontend/vite.config.js:/app/vite.config.js
      - ./frontend/index.html:/app/index.html
      - ./frontend/package.json:/app/package.json
''')

print("Docker Compose file was written under ai-web/docker-compose.yml.")

### Step 4: Git initialization
Git is initialized locally so changes can be tracked.

### FastAPI ↔ Vite AI workflow checklist

1. Scaffold FastAPI with a dedicated router for AI inference (e.g., `routers/inference.py`) and register it in `main.py`.
2. Define `Pydantic` models for request payloads (prompt text, model options) and responses (structured outputs, tokens, metadata).
3. Configure dependency-injected clients for external AI providers, loading keys from `.env` files via `python-dotenv` or container secrets.
4. Enable CORS middleware and document the route via FastAPI's autogenerated docs to confirm payload contracts.
5. In Vite, create API utility modules that read backend URLs from `import.meta.env` and call the FastAPI endpoints.
6. Build React components that manage prompt forms, loading indicators, streaming updates, and error toasts tied to those utilities.
7. Use Vite's dev server proxy to forward `/api` requests to FastAPI for local iteration, then validate the flow through Docker Compose.
8. Update production Dockerfiles or CI pipelines so backend and frontend environments receive the same `.env` configuration for AI model credentials.


```bash
cd ai-web
git init
git add .
git commit -m "Lab 1 scaffold"
```

## Validation / acceptance checks
```bash
# locally
docker compose up -d
curl http://localhost:8000/health
curl -X POST http://localhost:8000/echo -H 'Content-Type: application/json' -d '{"msg":"hello"}'
docker compose down
```
- HTTP 200 responses include status "ok" and the echoed payload while the stack runs in Docker.
- React development mode shows the described UI state without console errors.

## Homework / extensions
- A README entry is expanded to document backend and frontend start commands.
- A GitHub repository is connected for remote backups.